# Getting some hands-on experience with supervised machine learning

## Question 1

In this tutorial you will work with supervised machine learning. We will classify tweets into four categories namely: `normal`, `abusive`, `hateful`, and `spam`.

As you noted when reading the literature assigned for this and last week, there are a few steps that we need to take before we can use supervised machine learning. Namely:
* Collect data (in CS often texts, e.g., tweets)
* Develop a codebook and hand-code the data

In this tutorial, we focus on the actual machine learning part of the process. Hence, we will use a dataset that already has been coded by humans. It contains tweets and each tweet has a label indicating to which of four categories it belongs, namely normal, abusive, hateful, or spam. Hence, we skip the first two steps of the process described above.

Download the data for this exercise named "hatespeech_text_label_vote_RESTRICTED_100K.csv". These datafiles were retrieved from: https://www.dropbox.com/sh/4mapojr85a6sc76/AABYMkjLVG-HhueAgd0qM9kwa?dl=0

Using the examples from past lectures, can you write a script that opens each file and:
* Creates one list with the tweets
* Creates one list with the labels of the tweets

What could you do to check that this process went well? Can you explore the data a bit (i.e., by checking how often each label is present in the different datasets)?

## Question 2

Now that we have read in the data, we will proceed to the next step: Splitting our data into a training set and a test set. Luckily, scikit-learn has a function that can do so for us! Run the code presented in the next block to split up the dataset.
        
* What do these lines of code do?
* Do you know what the random_state part refers to? Why is this useful?


In [ ]:
from sklearn.model_selection import train_test_split

tweets_train, tweets_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=42)

## Question 3

On to the next step: Transforming the text into numbers, or setting up a vectorizer. Can you create some code that uses a count vectorizer on the texts that you read in? Hint: check out the example provided in the slides of previous weeks! Doing so, you will see that the stopwords are defined (as a built-in stop word list). Why is that done?

## Question 4

Now, let’s train a classifier and run it on the test data! Can you use the examples this week's lecture to train a Naïve Bayes classifer with our count vectorizer?

## Question 5

When you run the code you created for the previous question, you will see that it prints no output. How do you know if your code worked? Run the code presented in the next block (depending on how you named your labels, you may need to adjust the arguments).

Check out the documentation of the scikit learn package: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

What do the numbers in the output mean? What can you do with it?

In [ ]:
from sklearn.metrics import classification_report

print(y_pred[:10])
print(classification_report(y_test, y_pred))

## Question 6

Our last classifier was based on a count vectorizer using Naïve Bayes. Can you now train another classifier based on `Logistic Regression` and a `tf-idf vectorizer`?

## Question 7

As you saw in the article by Meppelink et al. (2021), we can try different combinations of these models (Naïve Bayes and Logistic Regression) and vectorizers (count and tf-idf). If you want to use Naïve Bayes and Logistic Regression as the models for a classifier, and a count vectorizer and a tf-idf vectorizer, how many classifiers could you then train? 

## Question 8

If we want to train multiple different classifiers, we could simply copy-paste the code used in the previous questions and adjust it for each of the classifiers. However, a cleaner approach is to write a function in which we define the specifics of each classifier. The code below does that.

In this code, we create a loop that trains each classifier by calling the function that is built in the first part of the code. 

Run the code below and compare it to the code that you wrote to train one classifier: Do you understand what is happening there?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

configs = [
    ("NB-count", CountVectorizer(min_df=5, max_df=.5), MultinomialNB()),
    ("NB-TfIdf", TfidfVectorizer(min_df=5, max_df=.5), MultinomialNB()),
    ("LR-Count", CountVectorizer(min_df=5, max_df=.5), LogisticRegression(solver="liblinear")),
    ("LR-TfIdf", TfidfVectorizer(min_df=5, max_df=.5), LogisticRegression(solver="liblinear"))
]

for name, vectorizer, classifier in configs:
    print(name)
    X_train = vectorizer.fit_transform(tweets_train)
    X_test = vectorizer.transform(tweets_test)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(classification_report(y_test, y_pred))


## Question 9

Check out the documentation of scikit learn (https://scikit-learn.org/stable/supervised_learning.html). Can you try to use other models and train a classifier with them? Can you merge this code into the code used in the previous question?

## Question 10

Based on the output that the classifier prints, what classifier performs the best? In your answer, consider:
* What information you need to identify the best classifier
* What metric you base your conclusion (i.e., precision, recall, accurcay, or F1-score) on and why

## Question 11

Let's say that you base your evaluation on the F1-score of the classifier. You can choose between the macro average and the weighted average of the F1-score. Check out the scikit learn documentation (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html). What F1-value (macro average or weighted average) would you select?

## Question 12

When looking at the classification report, you will see another column indicating values for something labelled 'support'.
Can you do some searching online and find out what 'support' is?

## Question 13

Two researchers want to use the classifier to distinguish between tweets that are spam or hateful and tweets that are not (either because they are normal or abusive). 
They are, however, not happy with the performance of the classifier when looking at the accuracy, precision, and recall for the spam category or the hateful category.
One of the researchers suggests to first recode the labels, so that all tweets that were annotated as spam receive a label 'spam' or 'hateful' are grouped together and all other tweets are grouped together as well.

What would the consequences be of doing so? What can you do to check your own answer? Try to recode the labels and see what happens!

## Question 14

For now, let's say that the classifier based on a count vectorizer and Logistic Regression is the one we prefer. We now want to use this model to predict the label for new data that we have not annotated (remember, this was the whole goal of SML)!

To do this, let’s save our classifier and our vectorizer to a file. If we don’t do this, we would need to re-train our model every time we want to use it. This is not so convenient, for example, we would always need to have our training data at hand. The code below shows you how to make a vectorizer and train a classifier (a repetition of what we did before to show you the whole process) and store them into a file.

In the code, you will see that both the classifier and the vectorizer are stored into a file. Why do you need to store both (why not just store the classifier only)?

In [ ]:
import pickle
import joblib

# Make a vectorizer and train a classifier
vectorizer=CountVectorizer(min_df=5, max_df=.5)
classifier=LogisticRegression(solver="liblinear")
X_train=vectorizer.fit_transform(tweets_train)
classifier.fit(X_train, y_train)

# Save them to disk
with open("myvectorizer.pkl",mode="wb") as f:
    pickle.dump(vectorizer, f)
with open("myclassifier.pkl",mode="wb") as f:
    joblib.dump(classifier, f)

# Later on, re-load this classifier and apply:
new_tweets = ["This Tweet is very shitty nasty mean and hateful", 
            "This is a very normal normal tweet.", 
            "2%^&GHJ &(&hrqjf3 click this link"]

with open("myvectorizer.pkl",mode="rb") as f:
    myvectorizer = pickle.load(f)
with open("myclassifier.pkl",mode="rb") as f:
    myclassifier = joblib.load(f)
    
new_features = myvectorizer.transform(new_tweets)
pred = myclassifier.predict(new_features)

for tweet, label in zip(new_tweets, pred):
    print(f"'{tweet}' is probably '{label}'.")